In [4]:
import pypcd

from sensor_msgs.msg import PointCloud2

import sqlite3
from rosidl_runtime_py.utilities import get_message
from rclpy.serialization import deserialize_message

import matplotlib.pyplot as plt

class BagFileParser():
    def __init__(self, bag_file):
        self.conn = sqlite3.connect(bag_file)
        self.cursor = self.conn.cursor()

        ## create a message type map
        topics_data = self.cursor.execute("SELECT id, name, type FROM topics").fetchall()
        self.topic_type = {name_of:type_of for id_of,name_of,type_of in topics_data}
        self.topic_id = {name_of:id_of for id_of,name_of,type_of in topics_data}
        self.topic_msg_message = {name_of:get_message(type_of) for id_of,name_of,type_of in topics_data}
        print(self.topic_msg_message)
    def __del__(self):
        self.conn.close()

    # Return [(timestamp0, message0), (timestamp1, message1), ...]
    def get_messages(self, topic_name):
        
        topic_id = self.topic_id[topic_name]
        print(topic_id)
        # Get from the db
        
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(self.cursor.fetchall())
        rows = self.cursor.execute("SELECT timestamp, data FROM messages WHERE topic_id = {}".format(topic_id)).fetchall()
        # Deserialise all and timestamp them
        return [ (timestamp,deserialize_message(data, self.topic_msg_message[topic_name])) for timestamp,data in rows]



if __name__ == "__main__":

        bag_file = '/home/uct-sea-ice/rosbag2_2022_11_24-12_37_51/data.db3'

        parser = BagFileParser(bag_file)

        pcds = parser.get_messages("/livox/lidar")
        print(len(pcds))


        for i in range(len(pcds)):
            pcd = pcds[i]
            print(pcd[0])
            pcd_data =pcd[1]
            pypcd.PointCloud
            pc = pypcd.PointCloud.from_msg(pcd_data)
            pc.save('foo.pcd', compression='binary_compressed')

ModuleNotFoundError: No module named 'rclpy._rclpy'
The C extension '/opt/ros/foxy/lib/python3.8/site-packages/rclpy/_rclpy.cpython-37m-x86_64-linux-gnu.so' isn't present on the system. Please refer to 'https://index.ros.org/doc/ros2/Troubleshooting/Installation-Troubleshooting/#import-failing-without-library-present-on-the-system' for possible solutions